In [1]:
import os
import glob
import pandas as pd
import re
import warnings
from saac.eval_utils import load_tda_data, load_occupation_data

warnings.filterwarnings('once')

In [2]:
'''
Midjourney has a file name limit of 100 characters,which cuts off a number of the submitted prompts in the file name
In order to merge the results with the generated prompt data
We will parse and extract the base prompt from the image_file column
'''
def load_image_analysis_results():
    eval_data_path = os.path.join('..','..', 'data','evaluation','raw')
    files = glob.glob(os.path.join(eval_data_path, '*.csv'))
    colnames = ['file_prompt','image_file','quadrant','bbox','skincolor','gender_woman','gender_man']
    results = pd.concat([pd.read_csv(fp,header=0, names=colnames)\
                     .assign(model =os.path.basename(fp).split('_')[0]) for fp in files],sort=False)
    base_prompt= []
    for row in results['image_file']:
        try:
            base_prompt.append(row.split('_')[1:-2])
        except:
            base_prompt.append('')
    results['base_prompt'] = base_prompt
    results['base_prompt'] = results['base_prompt'].apply(lambda x: ' '.join(x)+" photorealistic")
    results['base_prompt'] = results['base_prompt'].str.strip().str.rstrip()
    #Mapping gender detection values to single column
    noface= (results['skincolor'].isnull()).values
    unknown=  ((results['gender_woman']<=50.000000) & (results['gender_man']<=50.000000)).values
    woman= ((results['gender_woman']>50.000000) & (results['gender_man']<=50.000000)).values
    man= ((results['gender_man']>50.000000) & (results['gender_woman']<=50.000000)).values
    results['gender_detected_cat'] = 0
    results['gender_detected_cat'][noface]=1
    results['gender_detected_cat'][unknown]=2
    results['gender_detected_cat'][woman]=3
    results['gender_detected_cat'][man]=4
    gender_dict=  {
             1: 'no face',
             2: 'unknown',
             3: 'man',
             4: 'woman'
                }
    results['gender_detected_val'] = results['gender_detected_cat'].map(gender_dict)
    return results

In [3]:
def load_prompts():
    prompt_data_path = os.path.join('..','..', 'data','prompt_generation','processed')
    files = glob.glob(os.path.join(prompt_data_path, '*.csv'))
    cols = [0, 1, 5]
    colnames = ['full_prompt','tag','prompt_compound']
    prompts = pd.concat([pd.read_csv(fp,header=0,usecols= cols, names=colnames)for fp in files],sort=False)
    base_prompt =[]
    for line in prompts['full_prompt']:
        if '/imagine prompt:' in line:
            line = line.replace('/imagine prompt:','')
            line = line.replace(', photorealistic --s 625',' photorealistic')
            line = line.replace(',',' ')
            line = re.sub(' +',' ',line).strip().rstrip()
            base_prompt.append(line)
        else:
            base_prompt.append(line)
    prompts['base_prompt'] = base_prompt
    return prompts

In [4]:
def get_occupation_results():
    results = load_image_analysis_results()
    prompts = load_prompts()
    res_prompts = results.merge(prompts, right_on=['base_prompt'], left_on=['base_prompt'])
    occupations = load_occupation_data()
    occ_results = res_prompts.merge(occupations,right_on=['norm_title'],left_on=['tag'],how='inner')
    return occ_results

def get_tda_results():
    results = load_image_analysis_results()
    prompts = load_prompts()
    res_prompts = results.merge(prompts, right_on=['base_prompt'], left_on=['base_prompt'])
    tda_data = load_tda_data()
    tda_results = res_prompts.merge(tda_data,right_on=['tda'],left_on=['tag'],how='inner')
    return tda_results

In [5]:
tda = get_tda_results()
print(len(tda))

1440


C:\Users\vrozenbaum\AppData\Local\Temp\ipykernel_18160\4010911928.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['gender_detected_cat'][noface]=1


In [6]:
occ = get_occupation_results()
print(len(occ))

1440


C:\Users\vrozenbaum\AppData\Local\Temp\ipykernel_18160\4010911928.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['gender_detected_cat'][noface]=1


In [7]:
savepath='../../data/evaluation/processed/'

tda.to_csv(savepath+ 'TDA_Results.csv',index=False)
occ.to_csv(savepath+ 'Occupation_Results.csv',index=False)